Planificación usando Pyomo - Parte 1
===

**Juan David Velásquez Henao**  
jdvelasq@unal.edu.co   
Universidad Nacional de Colombia, Sede Medellín  
Facultad de Minas  
Medellín, Colombia

---

Haga click [aquí](https://github.com/jdvelasq/workshop-asocio-2018/tree/master/) para acceder al repositorio online.

Haga click [aquí](http://nbviewer.jupyter.org/github/jdvelasq/workshop-asocio-2018/tree/master/) para explorar el repositorio usando `nbviewer`. 

 ### Descripción del problema

Se desea realizar el planeamiento de la operacion de un sistema hidrotermico conformado por una planta hidraulica y otra termica. 
 
* **Planificacion:**    4 etapas. 


* **Planta hidraulica:**    
    * Vol. maximo (V*) = 100.
    * Caudal max. turbinado (Q*) = 50.
    * Factor conversion ($\rho$) = 1.
    * Aporte por etapa ($A_i$) = = {21, 15, 12, 18}.
    * Volumen inicial ($V_o$) = 75.  
    
    
* **Planta termica:**    
    * Generacion maxima (G*) = 45
    * Costo combustible (CC) = 15. 
 
 
* **Racionamiento:**   
    * Costo racionamiento (CR) = 1000 para todas las etapas. 
 
 
* **Demanda:**  50 para todas las etapas 
 

### Inicialización de Pyomo

In [1]:
%%writefile model.py

## Importacion de librerias
# from coopr.pyomo import *
from pyomo.environ import *

## Creando un modelo especifico Pyomo
model = ConcreteModel()


Writing model.py


### Definicion de variables

In [2]:
%%writefile -a model.py

## Energia racionada por etapa
model.R_1 = Var(within=NonNegativeReals)
model.R_2 = Var(within=NonNegativeReals)
model.R_3 = Var(within=NonNegativeReals)
model.R_4 = Var(within=NonNegativeReals)

## Generacion termica por etapa (solo hay una termica)
model.GT_1 = Var(within=NonNegativeReals)
model.GT_2 = Var(within=NonNegativeReals)
model.GT_3 = Var(within=NonNegativeReals)
model.GT_4 = Var(within=NonNegativeReals)

## Caudal turbinado por etapa (solo hay una hidro)

model.Q_1 = Var(within=NonNegativeReals)
model.Q_2 = Var(within=NonNegativeReals)
model.Q_3 = Var(within=NonNegativeReals)
model.Q_4 = Var(within=NonNegativeReals)
 
## Volumen final por etapa (solo hay una hidro)
model.V_1 = Var(within=NonNegativeReals)
model.V_2 = Var(within=NonNegativeReals)
model.V_3 = Var(within=NonNegativeReals)
model.V_4 = Var(within=NonNegativeReals)

## Vertimiento por etapa (solo hay una hidro)
model.S_1 = Var(within=NonNegativeReals)
model.S_2 = Var(within=NonNegativeReals)
model.S_3 = Var(within=NonNegativeReals)
model.S_4 = Var(within=NonNegativeReals)


Appending to model.py


### Funcion objetivo

La funcion objetivo busca minimizar el costo total del racionamiento mas el costo total de la generacion termica:


$$\text{minimize} ~ \sum_{p=1}^{P} ~\Bigg\{ CR * DEF_p + \sum_{t=1}^T CC_t * GT_{p,t} ~\Bigg\}$$







In [3]:
%%writefile -a model.py

## Definicion de la funcion objetivo
model.obj = Objective(expr = 1000*model.R_1 + 15*model.GT_1 + 
                             1000*model.R_2 + 15*model.GT_2 + 
                             1000*model.R_3 + 15*model.GT_3 + 
                             1000*model.R_4 + 15*model.GT_4)


Appending to model.py


### Demanda

Por cada periodo de tiempo $p$ debe cumplirse que:

$$ R_p + \Bigg\{ \sum_{h=1}^H \rho_h * Q_{hp} \Bigg\} + \Bigg\{ \sum_{t=1}^T GT_{t p} \Bigg\} = dem_p$$


In [4]:
%%writefile -a model.py

## una restriccion de demanda por cada etapa
model.demanda_1 = Constraint(expr = model.R_1 + model.GT_1 + model.Q_1 == 50) 
model.demanda_2 = Constraint(expr = model.R_2 + model.GT_2 + model.Q_2 == 50) 
model.demanda_3 = Constraint(expr = model.R_3 + model.GT_3 + model.Q_3 == 50) 
model.demanda_4 = Constraint(expr = model.R_4 + model.GT_4 + model.Q_4 == 50) 


Appending to model.py


### Continuidad de los volumenes de los embalses

Para el primer periodo, el embalse inicial $Vol_{h,0}$ es conocido, entonces:

$$Vol_{h,1} +Q_{h,1}+Ver_{h,1}=A_{h,1}+ Vol_{h,0}$$

In [5]:
%%writefile -a model.py

## una restricción de continuidad por cada etapa
model.continuidad_1 = Constraint(expr =  model.V_1 + model.Q_1 + model.S_1 == 96.) 
model.continuidad_2 = Constraint(expr = -model.V_1 + model.V_2 + model.Q_2 + model.S_2 == 15) 
model.continuidad_3 = Constraint(expr = -model.V_2 + model.V_3 + model.Q_3 + model.S_3 == 12) 
model.continuidad_4 = Constraint(expr = -model.V_3 + model.V_4 + model.Q_4 + model.S_4 == 18) 


Appending to model.py


### Caudal turbinado máximo

In [6]:
%%writefile -a model.py

## una restricción de caudal máximo por cada planta 
## (solo hay una) por cada etapa
model.qmax_1 = Constraint(expr = model.Q_1 <= 50 )  
model.qmax_2 = Constraint(expr = model.Q_2 <= 50 ) 
model.qmax_3 = Constraint(expr = model.Q_3 <= 50 ) 
model.qmax_4 = Constraint(expr = model.Q_4 <= 50 ) 


Appending to model.py


### Generacion térmica máxima

In [7]:
%%writefile -a model.py

## una restricción de generación máxima por 
## cada planta térmica y cada etapa

model.gtmax_1 = Constraint(expr = model.GT_1 <= 45 )
model.gtmax_2 = Constraint(expr = model.GT_2 <= 45 )
model.gtmax_3 = Constraint(expr = model.GT_3 <= 45 )
model.gtmax_4 = Constraint(expr = model.GT_4 <= 45 )


Appending to model.py


### Solucion

In [8]:
!pyomo solve --solver=glpk model.py

/bin/sh: pyomo: command not found


In [9]:
!cat results.yml

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 885.0
  Upper bound: 885.0
  Number of objectives: 1
  Number of constraints: 17
  Number of variables: 21
  Number of nonzeros: 36
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.008028984069824219
# ----------------------------------------------------------
#   Solution Information
# ---------------------------------

In [10]:
!rm model.py results.yml

Planificación usando Pyomo - Parte 1
===

**Juan David Velásquez Henao**  
jdvelasq@unal.edu.co   
Universidad Nacional de Colombia, Sede Medellín  
Facultad de Minas  
Medellín, Colombia

---

Haga click [aquí](https://github.com/jdvelasq/workshop-asocio-2018/tree/master/) para acceder al repositorio online.

Haga click [aquí](http://nbviewer.jupyter.org/github/jdvelasq/workshop-asocio-2018/tree/master/) para explorar el repositorio usando `nbviewer`. 